# 01 Pré-traitement des données

Ce notebook a pour objectif de charger, nettoyer et prétraiter les fichiers bruts du dataset KuaiRec afin de fournir des jeux de données prêts à l'emploi pour les étapes ultérieures du pipeline (construction des features, modélisation, etc.).

**Fichiers sources :**
- `big_matrix.csv` : interactions historiques (toutes interactions utilisateurs-vidéos)
- `small_matrix.csv` : sous-échantillon pour test et évaluation
- `social_network.csv` : graphe d’amitié des utilisateurs
- `item_categories.csv` : catégorisation des vidéos et traits (liste de features)
- `user_features.csv` : attributs des utilisateurs
- `item_daily_features.csv` : statistiques quotidiennes par vidéo

**Formats de sortie :** fichiers Parquet pour accélérer la réutilisation en mémoire créé dans un dossier 'preprocessed/'.

## 1. Import des dépendances

In [12]:
import pandas as pd
import os

## 2. Fonction de chargement des données

In [6]:
def load_data(data_dir="../data"):
    print("Loading datas...")
    # Chargement des tables d'interactions
    big_matrix = pd.read_csv(f"{data_dir}/big_matrix.csv")
    small_matrix = pd.read_csv(f"{data_dir}/small_matrix.csv")
    # Chargement du réseau social, conversion de la colonne friend_list en liste Python
    social_network = pd.read_csv(f"{data_dir}/social_network.csv")
    social_network["friend_list"] = social_network["friend_list"].map(eval)
    # Chargement des catégories et parsing des features
    item_categories = pd.read_csv(f"{data_dir}/item_categories.csv")
    item_categories["feat"] = item_categories["feat"].map(eval)
    # Chargement des features utilisateurs et journalières
    user_features = pd.read_csv(f"{data_dir}/user_features.csv")
    item_daily_feat = pd.read_csv(f"{data_dir}/item_daily_features.csv")
    print("All data loaded.")
    # Retour sous forme de dictionnaire
    return {
        "big_matrix": big_matrix,
        "small_matrix": small_matrix,
        "social_network": social_network,
        "item_categories": item_categories,
        "user_features": user_features,
        "item_daily_feat": item_daily_feat
    }

## 3. Exécution du chargement

In [7]:
dfs = load_data()
big = dfs["big_matrix"]
small = dfs["small_matrix"]
items = dfs["item_categories"]
users = dfs["user_features"]
daily = dfs["item_daily_feat"]
social = dfs["social_network"]

Loading datas...
All data loaded.


## 4. Nettoyage des données d'interactions
- Suppression des lignes ayant des clés manquantes (`user_id`, `video_id`, `timestamp`)
- Création d'une copie de `small` pour éviter les avertissements pandas

In [8]:
big = big.dropna(subset=["user_id", "video_id", "timestamp"])
small = small.dropna(subset=["user_id", "video_id", "timestamp"]).copy()

## 5. Conversion des timestamps
Les timestamps sont au format integer (secondes depuis Epoch). On les convertit en datetime pour faciliter les calculs temporels.

In [9]:
big["timestamp"] = pd.to_datetime(big["timestamp"], unit="s")
small.loc[:, "timestamp"] = pd.to_datetime(small["timestamp"], unit="s")

/tmp/ipykernel_148935/3233873522.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<DatetimeArray>
['2020-07-04 21:27:48.378000021', '2020-07-04 21:28:00.056999922',
 '2020-07-04 21:29:09.479000092', '2020-07-04 21:30:43.285000086',
 '2020-07-04 21:35:43.459000111', '2020-07-04 21:36:00.773000002',
 '2020-07-04 21:36:47.740999937', '2020-07-04 21:49:27.964999914',
 '2020-07-04 21:49:41.762000084', '2020-07-04 21:57:26.581000090',
 ...
 '2020-09-01 09:23:27.664000034', '2020-09-01 10:31:41.650000095',
 '2020-09-01 10:35:43.855000019', '2020-09-01 11:35:45.312999964',
 '2020-09-01 11:59:25.707999945', '2020-09-01 12:06:35.983999968',
 '2020-09-02 06:44:51.342000008', '2020-09-03 00:45:01.473999977',
 '2020-09-04 14:56:32.020999908', '2020-09-04 16:32:09.154000044']
Length: 4494578, dtype: datetime64[ns]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  small.loc[:, "timesta

## 6. Enrichissement par jointure de catégories

Pour chaque interaction, on attache la liste de features de la vidéo (`feat`) via une jointure à gauche.

In [10]:
big = big.merge(items[["video_id", "feat"]], on="video_id", how="left")
small = small.merge(items[["video_id", "feat"]], on="video_id", how="left")

## 7. Export vers Parquet

In [13]:
os.makedirs("preprocessed", exist_ok=True)

big.to_parquet("preprocessed/big_matrix.parquet", index=False)
small.to_parquet("preprocessed/small_matrix.parquet", index=False)
users.to_parquet("preprocessed/user_features.parquet", index=False)
items.to_parquet("preprocessed/item_categories.parquet", index=False)
daily.to_parquet("preprocessed/item_daily_features.parquet", index=False)
social.to_parquet("preprocessed/social_network.parquet", index=False)

print(f"Preprocessed files exported")

Preprocessed files exported
